In [5]:
from datetime import datetime
import random
random.seed(datetime.now().timestamp())
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
def random_roll():
    last_time = datetime.now().timestamp()
    while True:
        time_now = datetime.now().timestamp()
        if last_time != time_now:
            random.seed(time_now)
            last_time = time_now
        else:
            random.seed(random.random())
        yield random.random()
rnd = random_roll()

class Model:
    def __init__(self, config):
        self.attributes = ["yesterdays_barges","incoming_barges","unloaded_barges","day_end_barges"]
        self.config = config
        self.time = 0
        self.store = []
        self.incoming_index_range = len(self.config.incoming_distr)
        self.unloaded_index_range = len(self.config.unloading_distr)
        i_accumulator = 0
        u_accumulator = 0
        for i in range(self.incoming_index_range):
            i_accumulator += self.config.incoming_distr[i][1]
            self.config.incoming_distr[i].append(i_accumulator)
        for i in range(self.unloaded_index_range):
            u_accumulator += self.config.unloading_distr[i][1]
            self.config.unloading_distr[i].append(u_accumulator)
        self.yesterdays = 0
        self.incoming = 0
        self.unloaded = 0
    def random_gen(self):
        for i in range(self.incoming_index_range):
            if self.config.incoming_distr[i][2] >= next(rnd):
                self.incoming = self.config.incoming_distr[i][0]
                break
        for i in range(self.unloaded_index_range):
            if self.config.unloading_distr[i][2] >= next(rnd):
                self.unloaded = self.config.unloading_distr[i][0]
                break
    def time_increment(self):
        self.time += 1
        self.yesterdays += self.incoming - self.unloaded
        self.random_gen()
        days_end_count = self.yesterdays + self.incoming - self.unloaded
        if self.unloaded > day_end_count:
            self.unloaded += day_end_count
            day_end_count = 0
        return [
            self.yesterdays,
            self.incoming,
            self.unloaded,
            day_end_count
        ]
    def model(self, timespan = 3650):
        for _ in range(timespan):
            self.store.append(self.time_increment())
        df = pd.DataFrame(self.store, columns=self.attributes)
        print(df)
        return df

class Object:
    def __init__(self, **attributes):
        self.__dict__.update(attributes)

In [7]:
config = Object(
    incoming_distr = [
        [0, 0.13],
        [1, 0.17],
        [2, 0.15],
        [3, 0.25],
        [4, 0.2],
        [5, 0.1]],
    unloading_distr = [
        [1, 0.05],
        [2, 0.15],
        [3, 0.5],
        [4, 0.2],
        [5, 0.1]])

model = Model(config)
df = model.model(15)

    yesterdays_barges  incoming_barges  unloaded_barges  day_end_barges
0                   0                3                3               0
1                   0                1                1               0
2                   0                3                3               0
3                   0                2                2               0
4                   0                3                3               0
5                   0                1                1               0
6                   0                3                3               0
7                   0                3                3               0
8                   0                2                2               0
9                   0                4                2               2
10                  2                3                5               0
11                  0                0                0               0
12                  0                1                1         

In [8]:
df[["day_end_barges"]].describe()

,day_end_barges
count,15.000000
mean,0.200000
std,0.560612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,2.000000


In [9]:
df[["incoming_barges"]].describe()

,incoming_barges
count,15.000000
mean,2.066667
std,1.222799
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


In [10]:
df[["unloaded_barges"]].describe()

,unloaded_barges
count,15.00000
mean,2.00000
std,1.36277
min,0.00000
25%,1.00000
50%,2.00000
75%,3.00000
max,5.00000
